In [1]:
import logging
from IPython.display import JSON
import json
from winterapi import WinterAPI
from wintertoo.models import SummerFieldToO, SummerRaDecToO, WinterFieldToO, WinterRaDecToO

In [2]:
logging.getLogger("winterapi").setLevel("DEBUG")

In [3]:
winter = WinterAPI()

Our credentials from the previous notebook will still be active!

# Setting up a ToO

Now we are going to set up a ToO. There are two ways to trigger a ToO:
* Using a ra/dec value
* Using a field value

There are also two types of instruments to trigger: WINTER or SUMMER. 

That makes **4 different ToO objects to choose from*.

Let's go through the procedure for a Summer ToO using a field.

## Step 1: What can go into a ToO request?

For `winterapi`, we handle data using `pydantic`. We specify what types of data can be provided, what the conditions are, and what the default values are.

You can see all options in a human-readable way by converting the data model to a json schema:

In [4]:
print(json.dumps(WinterFieldToO.model_json_schema(), indent=2))

{
  "additionalProperties": false,
  "description": "Winter ToO Request with field",
  "properties": {
    "field_id": {
      "minimum": 1,
      "title": "Field ID",
      "type": "integer"
    },
    "filters": {
      "default": [
        "Y",
        "J",
        "Hs"
      ],
      "items": {
        "enum": [
          "dark",
          "Y",
          "J",
          "Hs"
        ],
        "type": "string"
      },
      "title": "Filters",
      "type": "array"
    },
    "target_priority": {
      "default": 50.0,
      "minimum": 0.0,
      "title": "Priority for target",
      "type": "number"
    },
    "t_exp": {
      "default": 30.0,
      "minimum": 1.0,
      "title": "Combined Exposure Time across dithers (s)",
      "type": "number"
    },
    "n_exp": {
      "default": 1,
      "minimum": 1,
      "title": "Number of dither sets",
      "type": "integer"
    },
    "n_dither": {
      "default": 1,
      "minimum": 1,
      "title": "Number of dithers",
      "type

There is a lot of information, but the most important line is right at the bottom. The only thing that is required is `field_id`. Everything else is optional.

We can initialise the ToO using a random field number:

In [5]:
too = WinterFieldToO(field_id=55)
too

WinterFieldToO(field_id=55, filters=['Y', 'J', 'Hs'], target_priority=50.0, t_exp=30.0, n_exp=1, n_dither=1, dither_distance=30.0, start_time_mjd=60341.0835267944, end_time_mjd=60342.07352679921, max_airmass=2.0)

As you can see, we got a full ToO request with sensible defaults. If we only wanted one filter rather than 3, we could specify this:

In [6]:
too = WinterFieldToO(field_id=55, filters=["J"])
too

WinterFieldToO(field_id=55, filters=['J'], target_priority=50.0, t_exp=30.0, n_exp=1, n_dither=1, dither_distance=30.0, start_time_mjd=60341.0835267944, end_time_mjd=60342.07352679921, max_airmass=2.0)

Pydantic also validates the data we enter, to make sure things are sensible. Here are some examples of things you can't do:

In [7]:
# Names a filter that isn't available
too = WinterFieldToO(field_id=55, filters=["V"])
too

ValidationError: 1 validation error for WinterFieldToO
filters.0
  Input should be 'dark', 'Y', 'J' or 'Hs' [type=literal_error, input_value='V', input_type=str]
    For further information visit https://errors.pydantic.dev/2.3/v/literal_error

In [8]:
# end time in the past
too = WinterFieldToO(field_id=55, end_time_mjd=58000.)
too

ValidationError: 1 validation error for WinterFieldToO
end_time_mjd
  Input should be greater than or equal to 60341.07352680047 [type=greater_than_equal, input_value=58000.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.3/v/greater_than_equal

In [9]:
# Chooses a nonsensical n_dither number
too = WinterFieldToO(field_id=55, n_dither=0)
too

ValidationError: 1 validation error for WinterFieldToO
n_dither
  Input should be greater than or equal to 1 [type=greater_than_equal, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.3/v/greater_than_equal

## Step 2 Choosing a ToO using ra/dec

If you know the field to observe, then you have all the information you need, because each grid has a unique ra/dec value for that field.
The API will look that value up, so you do not need to worry about it.

However, more often, if you are performing a ToO, it's because you know the ra/dec of a specific object.

In that case, we can use a Ra/Dec model. Here's one with Winter:

In [10]:
print(json.dumps(WinterRaDecToO.model_json_schema(), indent=2))

{
  "additionalProperties": false,
  "description": "Winter ToO Request with Ra/Dec",
  "properties": {
    "ra_deg": {
      "example": 180.0,
      "maximum": 360.0,
      "minimum": 0.0,
      "title": "Right ascension in decimal degrees",
      "type": "number"
    },
    "dec_deg": {
      "example": 0.0,
      "maximum": 90.0,
      "minimum": -90.0,
      "title": "Declination in decimal degrees",
      "type": "number"
    },
    "use_field_grid": {
      "default": true,
      "title": "boolean whether to select nearest field in grid for central ra/dec",
      "type": "boolean"
    },
    "filters": {
      "default": [
        "Y",
        "J",
        "Hs"
      ],
      "items": {
        "enum": [
          "dark",
          "Y",
          "J",
          "Hs"
        ],
        "type": "string"
      },
      "title": "Filters",
      "type": "array"
    },
    "target_priority": {
      "default": 50.0,
      "minimum": 0.0,
      "title": "Priority for target",
      "ty

In this case, we only need to provide the ra and dec:

In [11]:
too = WinterRaDecToO(ra_deg=300., dec_deg=51.)
too

WinterRaDecToO(ra_deg=300.0, dec_deg=51.0, use_field_grid=True, filters=['Y', 'J', 'Hs'], target_priority=50.0, t_exp=30.0, n_exp=1, n_dither=1, dither_distance=30.0, start_time_mjd=60341.0835267944, end_time_mjd=60342.07352679921, max_airmass=2.0)

**One important note: by default, a schedule with an ra/dec will be matched to the nearest field, and the exposure will be centered on that field RA/dec**
    
You can disable this behaviour by setting `use_field_grid` to false! 

In [12]:
too = WinterRaDecToO(ra_deg=300., dec_deg=51., use_field_grid=False)
too

WinterRaDecToO(ra_deg=300.0, dec_deg=51.0, use_field_grid=False, filters=['Y', 'J', 'Hs'], target_priority=50.0, t_exp=30.0, n_exp=1, n_dither=1, dither_distance=30.0, start_time_mjd=60341.0835267944, end_time_mjd=60342.07352679921, max_airmass=2.0)